In [1]:
import pandas as pd
import numpy as np
import datetime

/Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [15]:
df = pd.read_csv('/Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/data/raw/crypto/Bitstamp_BTCUSD.csv')
df['date'] = pd.to_datetime(df['t'],unit='ms')
df.head()

,q,p,s,t,d,side,buy_id,sell_id,date
0,0.001500,40383.25,btcusd,1623729486876,2021-06-15,0,1371727922049028,1371727921057793,2021-06-15 03:58:06.876
1,0.002050,40383.26,btcusd,1623729486876,2021-06-15,0,1371727922049028,1371727920283649,2021-06-15 03:58:06.876
2,0.185857,40375.12,btcusd,1623729517598,2021-06-15,0,1371728047882246,1371728022577152,2021-06-15 03:58:37.598
3,0.001500,40389.65,btcusd,1623729532894,2021-06-15,0,1371728110534658,1371728110309376,2021-06-15 03:58:52.894
4,0.001700,40389.66,btcusd,1623729532894,2021-06-15,0,1371728110534658,1371728103247872,2021-06-15 03:58:52.894


In [16]:
df =df.set_index('date')
price = df['p'].resample('60min').agg({'open': 'first',
                                           'high': 'max',
                                           'low': 'min',
                                           'close': 'last'
                                           })
volume = df['q'].resample('60min').agg({'volume':'sum'})

df = price.join(volume)

In [45]:
df.loc[df.index>datetime.datetime(2021,8,14,3,0,0)]

,open,high,low,close,volume
date,,,,,
2021-08-14 04:00:00,47556.17,47702.14,47471.02,47487.90,26.826744
2021-08-14 05:00:00,47487.90,47607.07,47456.61,47570.06,22.049277
2021-08-14 06:00:00,47555.30,47703.49,47532.90,47550.00,39.081748
2021-08-14 07:00:00,47550.00,47763.88,47526.00,47663.03,24.685183
2021-08-14 08:00:00,47661.99,47739.43,47560.47,47638.67,27.020115
...,...,...,...,...,...
2021-11-26 07:00:00,57585.33,57672.26,56661.09,56997.67,336.548185
2021-11-26 08:00:00,56960.23,56960.23,54377.58,55251.91,644.544758
2021-11-26 09:00:00,55218.27,55514.82,54619.36,54917.08,384.133860


In [17]:
close = df['close']
low = df['low']
high = df['high']
_open = df['open']

In [18]:
def get_Daily_Volatility(close,span0=20):
    # simple percentage returns
    df0=close.pct_change()
    # 20 days, a month EWM's std as boundary
    df0=df0.ewm(span=span0).std()
    df0.dropna(inplace=True)
    return df0

In [19]:
#set the boundary of barriers, based on 20 days EWM
daily_volatility = get_Daily_Volatility(close)
# how many days we hold the stock which set the vertical barrier
t_final = 10 
#the up and low boundary multipliers
upper_lower_multipliers = [2, 2]

In [20]:
close = close[daily_volatility.index]
low = low[daily_volatility.index]
high = high[daily_volatility.index]
_open = _open[daily_volatility.index]

In [21]:
def get_3_barriers():
    #create a container
    barriers = pd.DataFrame(columns=['days_passed', 'open',
              'close', 'high', 'low', 'vert_barrier', \
              'top_barrier', 'bottom_barrier'], \
               index = daily_volatility.index)
    for day, vol in daily_volatility.iteritems():
        days_passed = len(daily_volatility.loc \
                      [daily_volatility.index[0] : day])
        #set the vertical barrier 
        if (days_passed + t_final < len(daily_volatility.index) \
            and t_final != 0):
            vert_barrier = daily_volatility.index[
                                days_passed + t_final]
        else:
            vert_barrier = np.nan
        #set the top barrier
        if upper_lower_multipliers[0] > 0:
            top_barrier = close.loc[day] + close.loc[day] * \
                          upper_lower_multipliers[0] * vol
        else:
            #set it to NaNs
            top_barrier = pd.Series(index=close.index)
        #set the bottom barrier
        if upper_lower_multipliers[1] > 0:
            bottom_barrier = close.loc[day] - close.loc[day] * \
                          upper_lower_multipliers[1] * vol
        else: 
            #set it to NaNs
            bottom_barrier = pd.Series(index=prices.index)
        barriers.loc[day, ['days_passed', 'open', 'close', 'high', 'low',\
        'vert_barrier','top_barrier', 'bottom_barrier']] = \
         days_passed, _open.loc[day], close.loc[day], high.loc[day], low.loc[day], \
         vert_barrier, top_barrier, bottom_barrier

    # placeholder values
    barriers['barrier_touched'] = None
    barriers['y_pred'] = None
    barriers['prc_change'] = None
    barriers['barrier_touched_date'] = None
    
    return barriers


In [41]:
def get_labels(barriers):

    for i in range(len(barriers.index)-1):
        start = barriers.index[i]
        start_looking = barriers.index[i+1]
        end = barriers.vert_barrier[i]
        if pd.notna(end):
            # assign the initial and final price
            price_initial = barriers.close[start]
            price_final = barriers.close[end]
    # assign the top and bottom barriers
            top_barrier = barriers.top_barrier[i]
            bottom_barrier = barriers.bottom_barrier[i]
    #set the profit taking and stop loss conditons
            current_snapshot = barriers[start_looking: end]
            
            if current_snapshot.loc[current_snapshot['low'] <= bottom_barrier].shape[0] == 0:
                bottom_barrier_date = datetime.datetime(9999,1,1)
            else: 
                bottom_barrier_date = current_snapshot.loc[current_snapshot['low'] <= bottom_barrier].index.min()
            if current_snapshot.loc[current_snapshot['high'] >= top_barrier].shape[0] == 0:
                top_barrier_date = datetime.datetime(9999,1,1)
            else: 
                top_barrier_date = current_snapshot.loc[current_snapshot['high']  >= top_barrier].index.min()
                
            if bottom_barrier_date == top_barrier_date == datetime.datetime(9999,1,1):
                barriers['barrier_touched'][i] = 'vertical'
                barriers['prc_change'][i] = (price_final - price_initial)/price_final
                if barriers['prc_change'][i] > 0:
                    barriers['y_pred'][i] = 1
                else:
                    barriers['y_pred'][i] = 0
                barriers['barrier_touched_date'][i] = barriers['vert_barrier'][i]
            elif bottom_barrier_date < top_barrier_date:
                barriers['barrier_touched'][i] = 'bottom'
                barriers['y_pred'][i] = 0
                barriers['prc_change'][i] = (bottom_barrier - price_initial)/price_initial
                barriers['barrier_touched_date'][i] = bottom_barrier_date
            elif bottom_barrier_date > top_barrier_date:
                barriers['barrier_touched'][i] = 'top'
                barriers['y_pred'][i] = 1
                barriers['prc_change'][i] = (top_barrier - price_initial)/price_initial
                barriers['barrier_touched_date'][i] = top_barrier_date

    return barriers

In [37]:
i = 0
start_looking = barriers.index[i+1]
end = barriers.vert_barrier[i]
barriers[start_looking: end]

,days_passed,open,close,high,low,vert_barrier,top_barrier,bottom_barrier,barrier_touched,y_pred,prc_change,barrier_touched_date
date,,,,,,,,,,,,
2019-04-03 16:00:00,2,5034.37,5132.79,5152.0,5027.51,2019-04-04 03:00:00,5303.527518,4962.052482,top,1,0.033264,2019-04-03 20:00:00
2019-04-03 17:00:00,3,5133.79,5123.0,5190.0,5071.58,2019-04-04 04:00:00,5266.800026,4979.199974,top,1,0.028069,2019-04-03 20:00:00
2019-04-03 18:00:00,4,5125.13,5150.01,5188.01,5121.76,2019-04-04 05:00:00,5271.99832,5028.02168,top,1,0.023687,2019-04-03 20:00:00
2019-04-03 19:00:00,5,5137.25,5142.23,5176.03,5120.0,2019-04-04 06:00:00,5252.014529,5032.445471,top,1,0.02135,2019-04-03 20:00:00
2019-04-03 20:00:00,6,5146.89,5299.21,5321.0,5143.15,2019-04-04 07:00:00,5457.461819,5140.958181,bottom,0,-0.029863,2019-04-03 22:00:00
2019-04-03 21:00:00,7,5298.15,5182.3,5345.0,5161.0,2019-04-04 08:00:00,5371.83805,4992.76195,bottom,0,-0.036574,2019-04-03 23:00:00
2019-04-03 22:00:00,8,5182.3,5111.83,5265.13,5050.81,2019-04-04 09:00:00,5294.643802,4929.016198,bottom,0,-0.035763,2019-04-03 23:00:00
2019-04-03 23:00:00,9,5108.25,4978.23,5108.25,4800.5,2019-04-04 10:00:00,5169.819456,4786.640544,vertical,0,-0.002818,2019-04-04 10:00:00
2019-04-04 00:00:00,10,4972.83,5031.32,5036.44,4898.84,2019-04-04 11:00:00,5217.355492,4845.284508,vertical,0,-0.007338,2019-04-04 11:00:00


In [38]:
barriers = get_3_barriers()

In [39]:
barriers

,days_passed,open,close,high,low,vert_barrier,top_barrier,bottom_barrier,barrier_touched,y_pred,prc_change,barrier_touched_date
date,,,,,,,,,,,,
2019-04-03 15:00:00,1,5005.39,5010.0,5032.52,4997.4,2019-04-04 02:00:00,5055.023123,4964.976877,None,None,None,None
2019-04-03 16:00:00,2,5034.37,5132.79,5152.0,5027.51,2019-04-04 03:00:00,5303.527518,4962.052482,None,None,None,None
2019-04-03 17:00:00,3,5133.79,5123.0,5190.0,5071.58,2019-04-04 04:00:00,5266.800026,4979.199974,None,None,None,None
2019-04-03 18:00:00,4,5125.13,5150.01,5188.01,5121.76,2019-04-04 05:00:00,5271.99832,5028.02168,None,None,None,None
2019-04-03 19:00:00,5,5137.25,5142.23,5176.03,5120.0,2019-04-04 06:00:00,5252.014529,5032.445471,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-26 07:00:00,23225,57585.33,56997.67,57672.26,56661.09,NaN,57536.039502,56459.300498,None,None,None,None
2021-11-26 08:00:00,23226,56960.23,55251.91,56960.23,54377.58,NaN,56322.977003,54180.842997,None,None,None,None
2021-11-26 09:00:00,23227,55218.27,54917.08,55514.82,54619.36,NaN,55930.515552,53903.644448,None,None,None,None


In [42]:
barriers = get_labels(barriers)

In [14]:
barriers.reset_index(inplace=True)
barriers

,date,days_passed,open,close,high,low,vert_barrier,top_barrier,bottom_barrier,barrier_touched,y_pred,prc_change,barrier_touched_date
0,2019-04-04 12:00:00,1,4995.0,4907.94,5045.21,4778.59,2019-04-10 00:00:00,5051.3863,4764.4937,top,1,0.029227,2019-04-05 12:00:00
1,2019-04-05 00:00:00,2,4911.14,5004.01,5021.9,4883.83,2019-04-10 12:00:00,5193.553129,4814.466871,top,1,0.037878,2019-04-06 12:00:00
2,2019-04-05 12:00:00,3,5004.73,5039.73,5067.62,4951.8,2019-04-11 00:00:00,5193.627412,4885.832588,top,1,0.030537,2019-04-06 12:00:00
3,2019-04-06 00:00:00,4,5041.0,4998.15,5041.0,4963.26,2019-04-11 12:00:00,5139.216476,4857.083524,top,1,0.028224,2019-04-06 12:00:00
4,2019-04-06 12:00:00,5,4997.05,5041.39,5240.0,4900.0,2019-04-12 00:00:00,5170.176614,4912.603386,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,2021-11-24 00:00:00,1930,57583.91,56653.35,57750.2,56162.95,NaN,58769.644969,54537.055031,None,None,None,None
1930,2021-11-24 12:00:00,1931,56653.35,57185.06,57501.0,55894.62,NaN,59269.185781,55100.934219,None,None,None,None
1931,2021-11-25 00:00:00,1932,57169.19,58078.22,58499.97,57044.42,NaN,60192.111508,55964.328492,None,None,None,None
1932,2021-11-25 12:00:00,1933,58008.16,58990.14,59445.45,57953.97,NaN,61117.299521,56862.980479,None,None,None,None


In [43]:
barriers.loc[barriers['barrier_touched_date']==datetime.datetime(2021,7,4,21,0,0)]

,days_passed,open,close,high,low,vert_barrier,top_barrier,bottom_barrier,barrier_touched,y_pred,prc_change,barrier_touched_date
date,,,,,,,,,,,,
2021-07-04 11:00:00,19749,35616.94,35541.03,35621.74,35479.83,2021-07-04 22:00:00,35864.931926,35217.128074,top,1,0.009113,2021-07-04 21:00:00
2021-07-04 13:00:00,19751,35274.87,35354.35,35438.06,35274.87,2021-07-05 00:00:00,35707.434947,35001.265053,top,1,0.009987,2021-07-04 21:00:00
2021-07-04 14:00:00,19752,35355.63,35476.37,35520.08,35314.34,2021-07-05 01:00:00,35817.825795,35134.914205,top,1,0.009625,2021-07-04 21:00:00
2021-07-04 15:00:00,19753,35454.8,35513.31,35561.28,35401.08,2021-07-05 02:00:00,35838.442482,35188.177518,top,1,0.009155,2021-07-04 21:00:00
2021-07-04 16:00:00,19754,35513.36,35466.6,35689.91,35445.61,2021-07-05 03:00:00,35779.799344,35153.400656,top,1,0.008831,2021-07-04 21:00:00
2021-07-04 17:00:00,19755,35480.2,35563.33,35611.06,35403.46,2021-07-05 04:00:00,35864.646955,35262.013045,top,1,0.008473,2021-07-04 21:00:00
2021-07-04 18:00:00,19756,35544.62,35517.27,35544.62,35399.0,2021-07-05 05:00:00,35807.914603,35226.625397,top,1,0.008183,2021-07-04 21:00:00
2021-07-04 19:00:00,19757,35530.18,35572.8,35615.65,35501.19,2021-07-05 06:00:00,35850.127509,35295.472491,top,1,0.007796,2021-07-04 21:00:00
2021-07-04 20:00:00,19758,35565.98,35552.82,35572.08,35449.1,2021-07-05 07:00:00,35818.330346,35287.309654,top,1,0.007468,2021-07-04 21:00:00


In [194]:
barriers.drop(columns=['days_passed','vert_barrier','top_barrier','bottom_barrier','barrier_touched',
                      'prc_change','barrier_touched_date'], inplace=True)
barriers

,open,close,high,low
date,,,,
2021-01-02 00:00:00,731.99,731.99,743.0,717.0
2021-01-02 12:00:00,731.21,775.59,788.82,730.06
2021-01-03 00:00:00,775.4,830.51,832.64,769.95
2021-01-03 12:00:00,829.74,979.33,1014.7,827.23
2021-01-04 00:00:00,980.23,978.01,1169.97,890.0
...,...,...,...,...
2021-03-27 12:00:00,1680.08,1713.6,1732.5,1666.57
2021-03-28 00:00:00,1714.07,1702.97,1725.64,1698.1
2021-03-28 12:00:00,1702.4,1686.5,1713.65,1661.79


In [147]:
import ml_investing_wne.config as config
import pandas_ta as ta

In [150]:

def prepare_processed_dataset(plot=False, df=None, allow_null=False, features=True, ):
    '''

    :param plot:  flag whether to plot price evolution
    :param df: if no df passed, data will be loaded from folder passed in config
    :param allow_null: drop rows with nulls, effect of computing TA indictors (first obs)
    :param features: flag whether to add TA features
    :return: pandas dataframe
    '''

    # eliminate weekends
    df = df.loc[df['close'].notna()].copy()
    df['y_pred'] = df['close'].shift(-config.steps_ahead) / df['close']
    #df.ta.indicators()
    if features:
        MA = [3, 5, 10, 13, 20]
        for ma in MA:
            df.ta.sma(length=ma, append=True)
            df.ta.ema(length=ma, append=True)
            df.ta.variance(length=ma, append=True)

        df.ta.macd(append=True) # adds 3 columns
        df.ta.rsi(append=True) # adds 1 column
        df.ta.rsi(append=True, length=10)
        df.ta.rsi(append=True, length=6)
        #df.ta.rsi(length=10, append=True)
        #df.ta.rsi(length=20, append=True)
        df.ta.stoch(append=True) # adds 2 columns
        df.ta.willr(append=True) # adds 1 column
        df.ta.bbands(append=True) # adds 5 columns
        df.ta.xsignals(append=True) # does nothing
        # ta.roc doesn't work as expected
        # df.ta.roc(append=True)
        # df.ta.roc(length=12, append=True)
        # df.ta.roc(length=1, append=True)
        # df['y_pred'] = df['close'].shift(-config.steps_ahead) / df['close']
        df['roc_1'] = df['close'].shift(-1) / df['close']
        df['roc_1'] = df['roc_1'].shift(1)
        # df['y_roc_12'] = df['close'].shift(-12) / df['close']
        # df['y_roc_12'] = df['y_roc_12'].shift(12)
        df['datetime'] = df.index
        df['hour'] = df.datetime.dt.hour
        df['weekday'] = df.datetime.dt.weekday
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / df['hour'].max())
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / df['hour'].max())
        # I know there is an error but models were already trained with it
        # df['weekday_sin'] = np.sin(2 * np.pi * df['hour'] / df['weekday'].max())
        # df['weekday_cos'] = np.cos(2 * np.pi * df['hour'] / df['weekday'].max())
        df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / df['weekday'].max())
        df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / df['weekday'].max())
    # check of encoding
    # df.plot.scatter('hour_sin', 'hour_cos')
    # df.plot.scatter('weekday_sin', 'weekday_cos')
        df.drop(columns=['datetime'], axis=1, inplace=True)

    if not allow_null:
        df.dropna(inplace=True)
    output_directory = os.path.join(config.processed_data_path, config.currency)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    path = os.path.join(output_directory, '{}_processed_{}.csv'.format(config.currency, config.freq))
    df.to_csv(path)
    #logger.info('exported to {}'.format(path))

    return df

In [151]:
df = prepare_processed_dataset(df=barriers)

In [154]:
df

,days_passed,close,low,high,vert_barrier,top_barrier,bottom_barrier,barrier_touched,y_pred,prc_change,...,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos
date,,,,,,,,,,,,,,,,,,,,,
2021-01-18 12:00:00,34,1257.86,1210.48,1261.3,2021-01-24 00:00:00,1357.628937,1158.091063,top,1.136923,0.079316,...,1258.986932,4.199669,0.978239,1.026824,12,0,-2.449294e-16,1.0,0.000000,1.0
2021-01-19 00:00:00,35,1430.09,1251.49,1435.13,2021-01-24 12:00:00,1585.401713,1274.778287,bottom,0.957716,-0.108603,...,1432.140210,24.918641,0.993539,1.136923,0,1,0.000000e+00,1.0,0.866025,0.5
2021-01-19 12:00:00,36,1369.62,1333.08,1440.0,2021-01-25 00:00:00,1520.566564,1218.673436,bottom,0.9233,-0.110211,...,1467.100733,25.170168,0.702797,0.957716,12,1,-2.449294e-16,1.0,0.866025,0.5
2021-01-20 00:00:00,37,1264.57,1249.8,1408.28,2021-01-25 12:00:00,1414.678526,1114.461474,bottom,1.08954,-0.118703,...,1464.300802,23.655031,0.355178,0.9233,0,2,0.000000e+00,1.0,0.866025,-0.5
2021-01-20 12:00:00,38,1377.8,1235.12,1391.33,2021-01-26 00:00:00,1548.168732,1207.431268,bottom,0.896451,-0.123653,...,1475.218341,20.183814,0.639806,1.08954,12,2,-2.449294e-16,1.0,0.866025,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-22 00:00:00,159,1795.85,1753.34,1808.34,2021-03-27 12:00:00,1877.539733,1714.160267,bottom,0.936431,-0.045488,...,1847.406318,5.310712,0.460554,1.006874,0,0,0.000000e+00,1.0,0.000000,1.0
2021-03-22 12:00:00,160,1681.69,1654.3,1797.05,2021-03-28 00:00:00,1779.214541,1584.165459,top,1.002664,0.057992,...,1855.877870,10.002701,0.014751,0.936431,12,0,-2.449294e-16,1.0,0.000000,1.0
2021-03-23 00:00:00,161,1686.17,1651.3,1714.37,2021-03-28 12:00:00,1779.556554,1592.783446,bottom,0.990505,-0.055384,...,1843.127513,11.373167,0.208656,1.002664,0,1,0.000000e+00,1.0,0.866025,0.5


In [153]:
df.columns

Index(['days_passed', 'close', 'low', 'high', 'vert_barrier', 'top_barrier',
       'bottom_barrier', 'barrier_touched', 'y_pred', 'prc_change',
       'barrier_touched_date', 'SMA_3', 'EMA_3', 'VAR_3', 'SMA_5', 'EMA_5',
       'VAR_5', 'SMA_10', 'EMA_10', 'VAR_10', 'SMA_13', 'EMA_13', 'VAR_13',
       'SMA_20', 'EMA_20', 'VAR_20', 'MACD_12_26_9', 'MACDh_12_26_9',
       'MACDs_12_26_9', 'RSI_14', 'RSI_10', 'RSI_6', 'STOCHk_14_3_3',
       'STOCHd_14_3_3', 'WILLR_14', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0',
       'BBB_5_2.0', 'BBP_5_2.0', 'roc_1', 'hour', 'weekday', 'hour_sin',
       'hour_cos', 'weekday_sin', 'weekday_cos'],
      dtype='object')

In [32]:
i = 0
start = barriers.index[i]
end = barriers.vert_barrier[i]

# assign the initial and final price
price_initial = barriers.price[start]
price_final = barriers.price[end]

top_barrier = barriers.top_barrier[i]
bottom_barrier = barriers.bottom_barrier[i]

In [45]:
condition_sl = (barriers.price[start: end] <= bottom_barrier)

In [58]:
current_snapshot = barriers[start: end]
if current_snapshot.loc[current_snapshot['price'] <= bottom_barrier].shape[0] == 0:
    bottom_barrier_date = datetime.datetime(9999,1,1)
else: 
    bottom_barrier_date = current_snapshot.loc[current_snapshot['price'] <= bottom_barrier].index.min()
if current_snapshot.loc[current_snapshot['price'] >= top_barrier].shape[0] == 0:
    top_barrier_date = datetime.datetime(9999,1,1)
else: 
    top_barrier_date = current_snapshot.loc[current_snapshot['price']  >= top_barrier].index.min()
    
if bottom_barrier_date == top_barrier_date == datetime.datetime(9999,1,1):
    barrier_touched = 'vertical'
    out = 0.5
    prc_change = (price_final - price_initial)/price_final
elif bottom_barrier_date < top_barrier_date:
    barrier_touched = 'bottom'
    out = 0
    prc_change = (bottom_barrier - price_initial)/price_initial
elif bottom_barrier_date > top_barrier_date:
    barrier_touched = 'top'
    out = 1
    prc_change = (top_barrier - price_initial)/price_initial




In [64]:
bottom_barrier_date is not None

False

In [68]:
top_barrier_date > datetime.datetime(9999,1,1)

False

In [39]:
condition_pt = (barriers.price[start: end] >= top_barrier)
 

In [43]:
condition_pt.index.min()

Timestamp('2021-01-02 00:00:00', freq='720T')

In [22]:
barriers = get_3_barriers()

In [23]:
barriers

,days_passed,price,vert_barrier,top_barrier,bottom_barrier
date,,,,,
2021-01-02 00:00:00,1,731.99,2021-01-07 12:00:00,746.358994,717.621006
2021-01-02 12:00:00,2,775.59,2021-01-08 00:00:00,836.175668,715.004332
2021-01-03 00:00:00,3,830.51,2021-01-08 12:00:00,899.408616,761.611384
2021-01-03 12:00:00,4,979.33,2021-01-09 00:00:00,1132.45292,826.20708
2021-01-04 00:00:00,5,978.01,2021-01-09 12:00:00,1124.658649,831.361351
...,...,...,...,...,...
2021-03-27 12:00:00,170,1713.6,NaN,1820.285139,1606.914861
2021-03-28 00:00:00,171,1702.97,NaN,1804.107823,1601.832177
2021-03-28 12:00:00,172,1686.5,NaN,1782.3341,1590.6659


In [25]:
barriers['out'] = None

In [26]:
get_labels()


,days_passed,price,vert_barrier,top_barrier,bottom_barrier,out
date,,,,,,
2021-01-02 00:00:00,1,731.99,2021-01-07 12:00:00,746.358994,717.621006,1
2021-01-02 12:00:00,2,775.59,2021-01-08 00:00:00,836.175668,715.004332,1
2021-01-03 00:00:00,3,830.51,2021-01-08 12:00:00,899.408616,761.611384,1
2021-01-03 12:00:00,4,979.33,2021-01-09 00:00:00,1132.45292,826.20708,1
2021-01-04 00:00:00,5,978.01,2021-01-09 12:00:00,1124.658649,831.361351,1
...,...,...,...,...,...,...
2021-03-27 12:00:00,170,1713.6,NaN,1820.285139,1606.914861,None
2021-03-28 00:00:00,171,1702.97,NaN,1804.107823,1601.832177,None
2021-03-28 12:00:00,172,1686.5,NaN,1782.3341,1590.6659,None


In [27]:
barriers.out.value_counts()

 1.000000    81
-1.000000    39
-0.090531     1
 0.145254     1
 0.473042     1
 0.697410     1
 0.414155     1
 0.137835     1
 0.724293     1
 0.637629     1
 0.995086     1
 0.291507     1
-0.727920     1
-0.074864     1
 0.683572     1
-0.011122     1
-0.544817     1
-0.709282     1
-0.548763     1
-0.049052     1
 0.281469     1
-0.174836     1
-0.157553     1
-0.238658     1
 0.487141     1
 0.306123     1
 0.999117     1
 0.190592     1
-0.297797     1
 0.220718     1
-0.009806     1
-0.196266     1
 0.583777     1
 0.555649     1
 0.774714     1
 0.708151     1
 0.751981     1
 0.614527     1
 0.372732     1
 0.135912     1
-0.128201     1
 0.353473     1
 0.150781     1
 0.181258     1
-0.726399     1
Name: out, dtype: int64